In [ ]:
%load_ext autoreload
%autoreload 2

## Create Vocab for Cangjie

In [ ]:
import csv
reader = csv.reader(open("data/etl_952_singlechar_size_64/952_labels.txt"), 
                    delimiter=" "
                )

In [ ]:
list_cangjie = [line[-1].replace('zc', '-') for line in reader]

In [ ]:
list_cangjie = list_cangjie[1:]

In [ ]:
unique_chars = set()
for code in list_cangjie:
    for char in code:
        unique_chars.add(char)

In [ ]:
# add blank
vocab = sorted(list(unique_chars))
vocab.insert(0, "|")

In [ ]:
with open("cangjie_vocab.txt", "w") as f:
    for code in vocab:
        f.write(f"{code}\n")

In [ ]:
with open("cangjie_vocab.txt", "r") as f:
    vocab = f.read().split("\n")[:-1]

## Test Cangjie

In [ ]:
import csv
reader = csv.reader(open("data/etl_952_singlechar_size_64/952_labels.txt"), 
                    delimiter=" "
                )
next(reader, None)  # skip the headers

cangjie_dict = {
    int(line[0]): line[-1].replace('zc', '-')
    for line in reader
}

In [ ]:
cangjie_raws = list(cangjie_dict.values())[7]

In [ ]:
import torch
tokens = [vocab.index(token) for token in tokens]
tokens = torch.tensor(tokens, dtype=torch.long)

In [ ]:
tokens

In [ ]:
from utils import decode_cangjie

In [ ]:
decode_cangjie(tokens, torch.tensor([4]), raw=True)

In [ ]:
torch.full((1,), tokens.shape[0], dtype=torch.long).shape

In [ ]:
from models.resnext import seresnext50

In [ ]:
encoder = seresnext50()

In [ ]:
import torchvision.transforms as transforms
import PIL.Image as Image

test_image = "data/etl_952_singlechar_size_64/952_train/7/ETL8B-w2-n956_7.png"

image = Image.open(test_image)
transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ])
image = transformation(image)

In [ ]:
image.shape

In [ ]:
from torch.autograd import Variable

cls_out, ctc_out = encoder(image.unsqueeze(0))
preds_size = Variable(torch.IntTensor([ctc_out.size(0)] * 1))
ctc_out.shape

In [ ]:
preds_size

In [ ]:
ctc_out.max(2)

In [ ]:
_, preds = ctc_out.max(2)
preds = preds.transpose(1, 0).contiguous().view(-1)
preds

In [ ]:
_, preds = ctc_out.max(2)
preds = preds.transpose(1, 0).contiguous().view(-1)
sim_preds = decode_cangjie(preds.data, preds_size.data, raw=False)


In [ ]:
sim_preds

In [ ]:
import editdistance
editdistance.eval(sim_preds, cangjie_raws)

In [ ]:
for pred, target in zip(sim_preds, cpu_texts):
    if pred == target.lower():
        n_correct += 1

In [ ]:
b, c, h, w = output.size()
assert h == 1, "the height of output must be 1"
output = output.squeeze(2)
output = output.permute(2, 0, 1)  # [w, b, c]

# # rnn features
# output = self.rnn(output)

# # add log_softmax to converge output
# output = F.log_softmax(output, dim=2)

In [ ]:
output.shape

In [ ]:
gru = torch.nn.GRU(
    2048, 
    512, 
    2, 
    bidirectional = True
)
embedding = torch.nn.Linear(512 * 2, 28)

In [ ]:
rnn_out, _ = gru(output)
rnn_out.shape

In [ ]:
out = embedding(rnn_out)
out.shape

In [ ]:
out = F.log_softmax(out, dim=2)

In [ ]:
out.shape

In [ ]:
criterion = torch.nn.CTCLoss(
    blank=0, 
    reduction='mean', 
    zero_infinity=True
)

In [ ]:
tokens.shape

In [ ]:
torch.full((out.shape[1],), out.shape[0], dtype=torch.long).shape

In [ ]:
criterion(
    out,
    tokens.unsqueeze(0),
    torch.full((out.shape[1],), out.shape[0], dtype=torch.long),
    torch.full((1,), tokens.shape[0], dtype=torch.long)
)

In [ ]:
import torch.nn as nn
import torch.optim as optim
from conf import settings
from utils import get_network, get_training_dataloader, WarmUpLR, get_vocab

In [ ]:
from argparse import Namespace

args = Namespace(
    net="seresnext50",
    gpu=False,
    b=1,
    lr=0.1,
    resume=False,
    warm=1
)

In [ ]:
net = get_network(args)

In [ ]:
training_loader = get_training_dataloader(
    path="data/etl_952_singlechar_size_64/952_train",
    num_workers=4,
    batch_size=args.b,
    shuffle=True
)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)

In [ ]:
for batch_index, (images, labels) in enumerate(training_loader):
        if args.gpu:
            labels = labels.cuda()
            images = images.cuda()

        optimizer.zero_grad()
        
        outputs = net(images)
        print(outputs.shape)
        loss = loss_function(outputs, labels)
        # print(loss.item())
        
        loss.backward()
        optimizer.step()